#Data analyses

## Load in the data

In [1]:
import numpy as np, sys, scipy.stats, pandas as pd, os, os.path, csv, PythonAnalyses

import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline
pd.options.display.mpl_style = 'default'

Start = 0 #if I want to reload data from csvs = 1. If i want to load python pickle == 0

filename = 'AdditionalSingleton_TargetUncertain.csv'
csvName = 'AdditionalSingleton_TargetUncertain'

if sys.platform == 'linux2': #is this my linux laptop
    path = '/home/dan-laptop/Dropbox/TargetUncertainty/'
elif sys.platform == 'darwin': #is this my mac work comp    
    path = '/Users/danvatterott/Dropbox Encore/Dropbox/TargetUncertainty/'

os.chdir(path)

if Start == 1: #load in data from csv
    numfiles = len([name for name in os.listdir('.') if os.path.isfile(name) and name.endswith(".csv") and name[0].isdigit()])
    for partnum in xrange(1,numfiles+1): #this is a loop that goes through all participants
        filename = '%s_' % str(partnum) + csvName + '.csv'
        if partnum == 1: 
            df = pd.read_csv(filename)
        if partnum > 1: 
            df = df.append(pd.read_csv(filename))
    df.columns = map(str.lstrip, df.columns)
    df.to_pickle(csvName+'.pkl') #save this data frame all together
elif Start == 0: #load the data from a previously saved python file. should save time if not adding csv data. 
    df = pd.read_pickle(csvName+'.pkl')

/home/dan-laptop/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df[0:3] #take a quick look at the dataframe. 

,TarType,TarLoc,TarColorCode,DistCond,TarDistDist,DistLoc,DistLine,Resp,Block,ConstantTar,Acc,RT,Sub#
0,1,4,3,1,0,0,0,1,0,0,1,1544.0347,1
1,2,4,3,1,0,0,0,2,0,0,1,1136.9638,1
2,2,1,4,2,1,2,2,2,0,0,1,658.4343,1


SingletonCueMatch = 1 = color singleton at cued location/object
SingletonCueMatch = 2 = color singleton at uncued location/object
SingletonCueMatch = 0 = color singleton absent
BlockType = 1 = cross
BlockType = 2 = circle
BlockType = 3 = cardinal
BlockType = 4 = diagonal

## Data Trimming

data trimming. correct, slower than 300 ms and faster than 1500. 

In [ ]:
Trimmer = df['Block'] > 0 #not the practice block
Trimmer2 = df['Acc'] == 1 #not an error trial
Trimmer3 = df['RT'] > 300 #not a super fast response
PartTrim = (df['Sub#'] != 8)

DistCond = df[Trimmer & Trimmer2 & Trimmer3].groupby(['DistCond','Sub#']) 
RTTrim = DistCond['RT'].mean() + 2.5*DistCond['RT'].std(ddof=0)
subList = df.drop_duplicates(subset='Sub#')
TrimmerOld = [False]*len(df)
for sub in subList['Sub#']: #loop through every subject. 
    TrimmerA = (df.DistCond == 1) & (df['RT'] < RTTrim[1][sub]) & (df['Sub#'] == sub)
    TrimmerB = (df.DistCond == 2) & (df['RT'] < RTTrim[2][sub]) & (df['Sub#'] == sub)
    Trimmer4 = [any(tup) for tup in zip(TrimmerA, TrimmerB, TrimmerOld)] #combine those 2 RT trimmers. 
    TrimmerOld = Trimmer4

In [ ]:
totalTrialNum = np.size(df[PartTrim & Trimmer],0)
ex_counter = 0
Total=0
for sub in df['Sub#'].unique(): #loop through every subject. 
    TrimmerA = (df[Trimmer].DistCond == 1) & (df[Trimmer]['RT'] > RTTrim[1][sub]) & (df[Trimmer]['Sub#'] == sub) & (df[Trimmer].Acc==1)
    TrimmerB = (df[Trimmer].DistCond == 2) & (df[Trimmer]['RT'] > RTTrim[2][sub]) & (df[Trimmer]['Sub#'] == sub) & (df[Trimmer].Acc==1)
    TrimmerC = df[Trimmer]['RT'] < 300 & (df[Trimmer].Acc==1)
    Total+= (sum(TrimmerA)+sum(TrimmerB)+sum(TrimmerC))
Total = sum(TrimmerA) + sum(TrimmerB) + sum(TrimmerC) 
print Total
print totalTrialNum
Rt_ex = float(Total)/float(totalTrialNum)
print '%.9f' % Rt_ex
#measuring how many trials are excluded.

checking acc. looks good. everyone above 90%

In [ ]:
tableAcc = df[Trimmer].pivot_table(values='Acc', index='Sub#', aggfunc=np.mean)
tableAcc

## RT Data

In [ ]:
ConType = df['ConstantTar'] == 1
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','ConstantTarget')

In [ ]:
ConType = df['ConstantTar'] == 0
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','Changing Target')

In [ ]:
ConType = df['ConstantTar'] == 1
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
Diff_Constant = tableRT[2]-tableRT[1]
Absent1 = tableRT[1]
ConType = df['ConstantTar'] == 0
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
Diff_Change = tableRT[2]-tableRT[1]
Absent2 = tableRT[1]

print scipy.stats.ttest_ind(Diff_Constant, Diff_Change)
print scipy.stats.ttest_ind(Absent1, Absent2)

In [ ]:
plt.hist(Diff_Constant,histtype='step',linewidth=2);
plt.hist(Diff_Change,histtype='step',linewidth=2);

In [ ]:
ConType = df['ConstantTar'] == 1
tableRT3 = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=[], aggfunc=np.mean)
ConType = df['ConstantTar'] == 0
tableRT4 = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType].pivot_table(values='RT', index='Sub#', columns=[], aggfunc=np.mean)

scipy.stats.ttest_ind(tableRT3, tableRT4)

In [ ]:
df['Col_prevTrial'] = df['TarColorCode'].shift(periods=1)
df['Col_prevTrial2'] = df['TarColorCode'].shift(periods=2)

In [ ]:
df['Col_Rep'] = 0
df.loc[(df['Col_prevTrial'] == df['TarColorCode']), 'Col_Rep'] = 1
df.loc[(df['Col_prevTrial'] == df['TarColorCode']) & (df['Col_prevTrial2'] == df['TarColorCode']), 'Col_Rep1'] = 1

In [ ]:
ConType = df['ConstantTar'] == 0
TarRep = df['Col_Rep'] == 1
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & TarRep].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','Changing Target; Repeated TargetColor')

In [ ]:
ConType = df['ConstantTar'] == 0
TarRep = df['Col_Rep'] == 0
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & TarRep].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','Changing Target; Changed TargetColor')

In [ ]:
ConType = df['ConstantTar'] == 1
DTRIM = df['DistCond'] == 1
tableRT2A = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & DTRIM & ConType].pivot_table(values='RT', index='Sub#', columns=['Block'], aggfunc=np.mean)
DTRIM = df['DistCond'] == 2
tableRT2B = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & DTRIM & ConType].pivot_table(values='RT', index='Sub#', columns=['Block'], aggfunc=np.mean)
ConType = df['ConstantTar'] == 0
DTRIM = df['DistCond'] == 1
tableRT2C = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & DTRIM & ConType].pivot_table(values='RT', index='Sub#', columns=['Block'], aggfunc=np.mean)
DTRIM = df['DistCond'] == 2
tableRT2D = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & DTRIM & ConType].pivot_table(values='RT', index='Sub#', columns=['Block'], aggfunc=np.mean)

In [ ]:
xdata = df['Block'][PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4].unique()

Absent1  = np.mean(tableRT2A)
Present1 = np.mean(tableRT2B)
Absent2  = np.mean(tableRT2C)
Present2 = np.mean(tableRT2D)

fig = plt.figure(figsize=(15,8))
axes = fig.add_subplot(111)

axes.plot(xdata, Absent1, label='Singleton Absent:Constant Target'); 
axes.plot(xdata, Present1, label='Singleton Present:Constant Target')
axes.plot(xdata, Absent2,'--', label='Singleton Absent:Changing Target'); 
axes.plot(xdata, Present2,'--', label='Singleton Present:Changing Target')
axes.legend()
axes.set_ylim(600,900)
axes.set_xlim(0,np.max(xdata)+1);

In [ ]:
ConType = df['ConstantTar'] == 1
GreenShade = df['TarColorCode'] == 1 #this is the darkest green
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & GreenShade].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','ConstantTarget; Shade 1')

In [ ]:
ConType = df['ConstantTar'] == 1
GreenShade = df['TarColorCode'] == 2 #this is the 2nd darkest green
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & GreenShade].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','ConstantTarget; Shade 1')

In [ ]:
ConType = df['ConstantTar'] == 1
GreenShade = df['TarColorCode'] == 3 #this is the 2nd brightest green
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & GreenShade].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','ConstantTarget; Shade 1')

In [ ]:
ConType = df['ConstantTar'] == 1
GreenShade = df['TarColorCode'] == 4 #this is the brightest green
tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & GreenShade].pivot_table(values='RT', index='Sub#', columns=['DistCond'], aggfunc=np.mean)
PythonAnalyses.dataTableRT(tableRT)

PythonAnalyses.Test_and_Plot_PairwiseComparison(tableRT, 600, 1000, 'Singleton Condition','ConstantTarget; Shade 1')

In [ ]:
ConType = df['ConstantTar'] == 1
TrialT = df['DistCond'] == 1
RTs1 = df[Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & TrialT & ConType]['RT'].quantile(np.arange(0.1,1.1,0.1))
TrialT = df['DistCond'] == 2
RTs2 = df[Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & TrialT & ConType]['RT'].quantile(np.arange(0.1,1.1,0.1))
ConType = df['ConstantTar'] == 0
TrialT = df['DistCond'] == 1
RTs3 = df[Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & TrialT & ConType]['RT'].quantile(np.arange(0.1,1.1,0.1))
TrialT = df['DistCond'] == 2
RTs4 = df[Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & TrialT & ConType]['RT'].quantile(np.arange(0.1,1.1,0.1))
xdata = np.arange(1,11,1)

Singleton_Absent1  = RTs1
Singleton_Present1 = RTs2
Singleton_Absent0  = RTs3
Singleton_Present0 = RTs4

fig = plt.figure(figsize=(15,8))
axes = fig.add_subplot(111)

axes.plot(xdata, Singleton_Absent1, label='Singleton Absent1'); 
axes.plot(xdata, Singleton_Present1, label='Singleton Present1')
axes.plot(xdata, Singleton_Absent0, label='Singleton Absent0'); 
axes.plot(xdata, Singleton_Present0, label='Singleton Present0')
axes.legend(loc=2)
axes.set_ylim(500,2000)
axes.set_xlim(0,11);

In [ ]:
def print_tests(series1, series2):
    normTest = scipy.stats.shapiro(series1-series2)
    Test = scipy.stats.ttest_rel(series1,series2)
    print '\t normality test. Test value: %s P-value: %s' % (str(np.round(normTest[0],2)), str(np.round(normTest[1],4)))
    print '\t T-test. Test value: %s P-value: %s' % (str(np.round(Test[0],2)), str(np.round(Test[1],4)))
    
def GroupCapAnalysis_and_Plot(tableRT2, CIs):
    PlotFrame = pd.DataFrame([tableRT2.mean()])
    PlotFrame.columns = ['Incongruent', 'Congruent']
    PlotFrame2 = pd.DataFrame()
    PlotFrame2['Congruent'] = pd.DataFrame([CIs])
    PlotFrame2['Incongruent'] = pd.DataFrame([CIs])
    PlotFrame.plot(ylim = [500, 900], yerr=PlotFrame2, kind='bar')
    plt.xticks(range(1), ['Congruency of Singleton Line'], rotation=0);

    disp_tab = np.round(tableRT2,2)
    disp_tab['Incong-cong'] = disp_tab[False] - disp_tab[True]
    print disp_tab

    print '---------------------------------'
    print 'Mean Congruent: %s' % (str(round(np.mean(tableRT2[True]),2)))
    print 'Mean Incongruent: %s' % (str(round(np.mean(tableRT2[False]),2)))
    print 'Incongruent - Congruent: %s' % (str(round(np.mean(tableRT2[False])-np.mean(tableRT2[True]),2)))
    print '---------------------------------'
    print 'Neutral vs Singleton Distractor'
    print_tests(tableRT2[True],tableRT2[False])
    
def SimpleComparisonCI(table):
    import scipy.stats, math
    ttest = scipy.stats.ttest_rel(table[True], table[False])
    MeanDiff_byT = abs((table[False].mean()-table[True].mean())/ttest[0])
    CI = MeanDiff_byT*scipy.stats.t.isf(0.025, len(table)-1)*math.pow(2,0.05)/2
    return CI

In [ ]:
ConType = df['ConstantTar'] == 1
TrialT = df['DistCond'] == 2
df['Cong'] = (df['TarType'] == df['DistLine'])

tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & TrialT].pivot_table(values='RT', index='Sub#', columns=['Cong'], aggfunc=np.mean)
CIs = SimpleComparisonCI(tableRT)
GroupCapAnalysis_and_Plot(tableRT, CIs)

In [ ]:
ConType = df['ConstantTar'] == 0
TrialT = df['DistCond'] == 2
df['Cong'] = (df['TarType'] == df['DistLine'])

tableRT = df[PartTrim & Trimmer & Trimmer2 & Trimmer3 & Trimmer4 & ConType & TrialT].pivot_table(values='RT', index='Sub#', columns=['Cong'], aggfunc=np.mean)
CIs = SimpleComparisonCI(tableRT)
GroupCapAnalysis_and_Plot(tableRT, CIs)